In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import inspect
currentdir = os.path.dirname(os.path.abspath(
    inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(1, parentdir+'/src')
import torch
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
import time
from tqdm import trange
import matplotlib.pyplot as plt
import matplotlib

In [3]:
import gym
import pybullet as p
import stage.envs
from stage.tasks.kuka import KukaPETS
from stage.utils.nn import use_gpu
use_gpu()

In [4]:
savepath = parentdir + '/data/kuka/'

In [5]:
kuka = KukaPETS(render=False, 
                   action_parameterization='pd')
nq, nv, nu, nx, na = kuka.nq, kuka.nv, kuka.nu, kuka.nx, kuka.na

In [ ]:
_ = kuka.learn(30, verbose=True)
kuka.save_training_data(savepath+'data_pd_reg')

Network training: 100%|██████████| 10/10 [00:00<00:00, 26.44epoch(s)/s, Training loss MSE=52.381]  


Iteration:  0
avg. decision time:  0.004681695302327474
obs. reward:  -634.54297
act. reward:  -4.4964976


Network training: 100%|██████████| 10/10 [00:01<00:00,  8.26epoch(s)/s, Training loss MSE=21.913654]


Iteration:  1
avg. decision time:  0.13533103307088215
obs. reward:  -751.82776
act. reward:  -0.61913955


Network training: 100%|██████████| 10/10 [00:01<00:00,  5.47epoch(s)/s, Training loss MSE=17.528263]


...

Iteration:  21
avg. decision time:  0.1355211019515991
obs. reward:  -306.49268
act. reward:  -0.35888413


Network training: 100%|██████████| 10/10 [00:05<00:00,  1.68epoch(s)/s, Training loss MSE=1.0248482]


Iteration:  22
avg. decision time:  0.13502832889556884
obs. reward:  -276.79282
act. reward:  -0.04342739


Network training: 100%|██████████| 10/10 [00:08<00:00,  1.20epoch(s)/s, Training loss MSE=0.9876633]


Iteration:  23
avg. decision time:  0.13509948253631593
obs. reward:  -250.2057
act. reward:  -0.04912479


In [ ]:
traj, log = kuka.perform(kuka.goal)
act_seq = traj[:, 14:28]
initial_obs = traj[0, :14]
final_obs = traj[-1, :14]

In [ ]:
n_sample = 50
traj_pred = kuka.dynamics.unroll(initial_obs, 
                                 act_seq, 
                                 n_sample)

predicted_err = torch.norm(traj_pred[-1, :, :]-final_obs.expand(n_sample, -1), p=2, dim=1)
print (predicted_err.mean())
print (predicted_err.std())

In [ ]:
traj_pred_mean = torch.mean(traj_pred, dim=1)
traj_pred_std = torch.std(traj_pred, dim=1)
traj_pred_mean = traj_pred_mean.detach().cpu().numpy()
traj_pred_std = traj_pred_std.detach().cpu().numpy()
traj = traj.detach().cpu().numpy()

In [ ]:
font = {'family' : 'serif',
        'size'   : 22}

matplotlib.rc('font', **font)
np.set_printoptions(precision=3, linewidth=200, suppress=True)

In [ ]:
# traj_pred_mean = np.load('traj_pred_mean.npy')
# traj_pred_std = np.load('traj_pred_std.npy')
# traj = np.load('traj.npy')

In [ ]:
d = 1
dt = 0.01
task_horizon = 150
fig, ax = plt.subplots(d, figsize=(10, d * 6))
t = np.arange(0.0, task_horizon*dt, dt)

i = 1
desired = kuka.step_cost.desired[i].detach().cpu().numpy()*np.ones(len(t))

ax.plot(t, traj_pred_mean[:, i], lw=4, color='b', label='predicted mean')
ax.plot(t, traj[:, i], lw=4, ls='-.', color='orange', label='actual')

ax.plot(t, desired, lw=2, color='k', ls='-.', label='desired')

lb = traj_pred_mean - traj_pred_std
ub = traj_pred_mean + traj_pred_std
ax.fill_between(t, lb[:, i], ub[:, i], facecolor='blue',
            alpha=0.2, label='one-std region')
_ = ax.grid()
_ = ax.set_ylabel('Joint angle [rad]')
_ = ax.set_xlabel('Time [s]')
# _ = ax.set_xlim([0,0.5])
# _ = ax.set_ylim([-3.2, 3.2])

ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3),
             ncol=2, fancybox=True, shadow=True)
# for i in range(d):
#     ax[i].plot(t, traj[:, i], lw=4, color='orange', label='actual')
#     ax[i].plot(t, traj_pred_mean[:, i], lw=4, color='b', label='predicted mean')
#     lb = traj_pred_mean - traj_pred_std
#     ub = traj_pred_mean + traj_pred_std
#     ax[i].fill_between(t, lb[:, i], ub[:, i], facecolor='blue',
#                 alpha=0.2)
# fig.savefig('prediction_with_reg_150steps' + '.png', bbox_inches='tight')